In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [128]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [129]:
from crewai import Agent, Task, Crew
import os

In [143]:
os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"
os.environ["OPENAI_MODEL_NAME"] = "llama3-70b-8192"
os.environ["SERPER_API_KEY"] = "625727de4472e02e28369accecac92e0800c1a39"
os.environ["OPENAI_API_KEY"] = "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"

In [144]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [145]:
from crewai import Agent, Task, Crew

In [152]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./resume.md')
semantic_search_resume = MDXSearchTool(mdx='./resume.md')

Inserting batches in chromadb:   0%|          | 0/1 [00:00<?, ?it/s]


In [154]:
from IPython.display import Markdown, display
display(Markdown("./resume.md"))


# Abdullah Bin Altaf
- Email: abdullahkoraal@gmail.com
- Phone: +923310446605

## Profile
Abdullah Bin Altaf is an AI engineer specializing in Generative AI and Agentic AI, with expertise in coding and no-code platforms. Passionate about creating innovative AI driven solutions, building autonomous AI agents, and leveraging generative models to solve complex challenges and advance the field of artificial intelligence. 
## Work History

### AlphaSquad: AI Intern — Feb 2025 - Present
- Created AI workflow diagrams, built a RAG application, performed data analytics, developed a neural network for 
handwritten digit recognition, and worked on ETL processes.

### Rozgar: Generative AI Engineer — Oct 2024 - Present
- Volunteering to develop a platform that shows job posts and ATS scores of resumes by scraping job listings, passing 
them through LLMs to extract key information, and storing the results in a database. 

### Headstarter: Software Engineering Felow — July 2024 – Sep 2024
- Built and deployed 5 AI projects in 5 weeks, collaborated with a team to develop a customer support agent, a SaaS 
product, and Rate My Professor, utilizing technologies like GPT-3.5-turbo, Stripe, Pinecone, Next.js and more. 

## Education

### B.Sc. in Computer Science
Abasyn University - Bachelor’s degree

Abdullah Bin Altaf is an ideal candidate for junior roles, particularly in companies seeking junior AI developers with a robust blend of technical and strategic expertise.


In [155]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [156]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [157]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [158]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [159]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [160]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [161]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [162]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [163]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [164]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/abdullah-k18',
    'personal_writeup': """Abdullah Bin Altaf is an AI engineer specializing in Generative AI and Agentic AI,
    with expertise in coding and no-code platforms. Passionate about creating innovative AI driven solutions,
    building autonomous AI agents, and leveraging generative models to solve complex challenges
    and advance the field of artificial intelligence."""
}

In [165]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/abdullah-k18) URLs, and personal write-up (Abdullah Bin Altaf is an AI engineer specializing in Generative AI and Agentic AI, 
    with expertise in coding and no-code platforms. Passionate about creating innovative AI driven solutions,
    building autonomous AI agents, and leveraging generative models to solve complex challenges
    and advance the field of artificial intelligence.). Utilize tool

Exception in thread Thread-34 (_execute):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/dist-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/usr/local/lib/python3.11/dist-packages/crewai/agents/executor.py", line 124, in _call
    next_step_output = self._take_next_step(
           

Thought: I need to find a specific job posting on the careers page of AI Fund.

Action: Read website content
Action Input: {"website_url": "https://careers.aifund.ai/jobs"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to find a specific job posting on the careers page of AI Fund.

Action: Search the internet
Action Input: {"search_query": "AI Fund job posting senior full stack engineer"} 


Search results: Title: Ai Fund Senior Fullstack Engineer Job in San Francisco, CA
Link: https://www.ziprecruiter.com/c/AI-Fund/Job/Senior-Fullstack-Engineer/-in-San-Francisco,CA?jid=196837a1d09c91b6
Snippet: Easy 1-Click Apply Ai Fund Senior Fullstack Engineer Other ($130800 - $186200) job opening hiring now in San Francisco, CA 94199. Don't wait - apply now!
---
Title: Jobs at AI Fund Companies | AI Fund
Link: https://careers.aifund.ai/
Snippet: Sales Development Representative (SDR) · Compliance and Security Manager · Se

Exception in thread Thread-33 (_execute):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/dist-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/usr/local/lib/python3.11/dist-packages/crewai/agents/executor.py", line 124, in _call
    next_step_output = self._take_next_step(
           

 

AI Fund hiring Full-Stack Engineer in San Francisco Bay Area | LinkedIn
 Skip to main content
LinkedIn
 Full-Stack Engineer in Las Vegas, NV
Expand search
This button displays the currently selected search type. When expanded it provides a list of search options that will switch the search inputs to match the current selection. Jobs
 People
 Learning
Clear text
Clear text
Clear text
Clear text
Clear text
 Join now
 Sign in
Full-Stack Engineer
AI Fund
San Francisco Bay Area
 Apply
Join or sign in to find your next job
Join to apply for the Full-Stack Engineer role at AI Fund
 Not you?
 Remove photo
First name
Last name
Email
Password (6+ characters)
 By clicking Agree & Join, you agree to the LinkedIn User Agreement, Privacy Policy and Cookie Policy.
 Continue
 Agree & Join
or
 You may also apply directly on company website.
Security verification
 Already on LinkedIn? Sign in
Full-Stack Engineer
 AI Fund
 San Francisco Bay Area
 2 weeks ago
 31 applicants
 See who AI Fund has hired f

In [166]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Abdullah Bin Altaf
- Email: abdullahkoraal@gmail.com
- Phone: +923310446605

## Profile
Highly skilled AI engineer with expertise in Generative AI and Agentic AI, leveraging coding and no-code platforms to craft innovative AI-driven solutions. Proven track record of building autonomous AI agents and applying generative models to tackle complex challenges and advance artificial intelligence. Passionate about creating cutting-edge AI solutions that drive business growth and improvement.

## Work History

### AlphaSquad: AI Intern — Feb 2025 - Present
Utilized AI workflow diagrams to streamline processes, developed a RAG application to enhance data analytics, and designed a neural network for handwritten digit recognition. Additionally, worked on ETL processes to optimize data management.

### Rozgar: Generative AI Engineer — Oct 2024 - Present
Developed a platform that integrates AI-driven job posting and ATS scoring by scraping job listings, extracting key information using LLMs, and storing results in a database. This showcases my expertise in building innovative AI solutions.

### Headstarter: Software Engineering Fellow — July 2024 – Sep 2024
Collaborated with a team to develop a customer support agent, a SaaS product, and Rate My Professor, utilizing cutting-edge technologies like GPT-3.5-turbo, Stripe, Pinecone, and Next.js. Successfully built and deployed 5 AI projects in 5 weeks, demonstrating my ability to work efficiently and effectively.

## Education

### B.Sc. in Computer Science
Abasyn University - Bachelor’s degree

## Skills:

* Programming languages: Python, Java, R
* AI frameworks: TensorFlow, PyTorch
* AI models: GPT-3.5-turbo
* Technologies: Stripe, Pinecone, Next.js
* Expertise: Generative AI, Agentic AI, coding and no-code platforms
* Proficient in: AI workflow diagrams, RAG applications, data analytics, neural networks, ETL processes, scraping job listings, and extracting key information using LLMs.

This rewritten resume effectively highlights the candidate's qualifications and experiences relevant to the AI engineer position, making it easier for potential employers to identify their strengths and expertise.

In [167]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for AI Engineer Position**

**Introduction:**
As an AI Engineer, you will be responsible for developing and deploying personalized large language models (LLMs) to empower search and recommendation systems for a variety of applications. You will also conduct applied research on new LLM techniques for concrete product use cases, work with collaborating teams to integrate LLMs into our products and services, and contribute to the direction and roadmaps of Search and discover personalization.

**Technical Skills:**

* Can you elaborate on your experience with Generative AI and Agentic AI, and how you've applied these skills in previous roles?
* How do you approach building autonomous AI agents, and what challenges have you faced in this area?
* Can you walk me through your process for developing AI workflow diagrams and building RAG applications?
* How do you stay up-to-date with the latest developments in AI and machine learning, and how do you see these technologies evolving in the future?
* Can you give an example of a complex challenge you've faced in a previous role, and how you leveraged generative models to solve it?

**Project Experience:**

* Can you tell me more about your experience with building a platform that integrates AI-driven job posting and ATS scoring by scraping job listings, extracting key information using LLMs, and storing results in a database?
* How did you utilize AI workflow diagrams to streamline processes, develop a RAG application to enhance data analytics, and design a neural network for handwritten digit recognition in your previous role?
* Can you walk me through your experience with ETL processes to optimize data management?

**Education and Certifications:**

* Can you tell me more about your Bachelor's degree in Computer Science from Abasyn University?
* Do you have any relevant certifications or training in AI, machine learning, or data science?

**Soft Skills:**

* Can you give an example of a time when you had to work with a cross-functional team to develop a customer support agent, a SaaS product, and Rate My Professor?
* How do you handle conflicts or disagreements with team members, and what strategies do you use to resolve them?
* Can you tell me about a project you led and the results you achieved?

**Final Thoughts:**
Remember to highlight your technical skills, project experience, education, and soft skills throughout the interview. Show enthusiasm for the role and the company, and be prepared to ask questions about the position and the organization. Good luck!